## Homework 5: Mixed-effects models

In this homework we will continue analyzing the lexical decision data from Homework 4. 

We will be using the files `ldt_trials.csv` and `word_properties.csv` in the `data` folder. These files contain lexical decision reaction times for each participant and each word (`ldt_trials.csv`) and information about various word properties such as word length, word frequency, concreteness and emotional valence (`word_properties.csv`). These data were obtained from the English Lexicon Project: https://elexicon.wustl.edu/index.html

#### Part 1: Loading and organizing the data

Load the two datasets and repeat all preprocessing steps from Homework 4. You should end up with a dataset of 35978 rows.

In [ ]:
library(tidyverse)

ldt_trials <- read.csv('data/ldt_trials.csv')
word_properties <- read.csv('data/word_properties.csv')

df1 <- left_join(ldt_trials, word_properties, by=c("D_word"="Word"))  %>% 
    filter(!is.na(Length))

head(df1)
nrow(df1)

Transform the Sub_ID variable into a factor using the `as.factor()` command:

In [ ]:
# insert code here

#### Part 2: Plot the effect of Length on RTs separately for each participant

In Homework 4 we saw that lexical decision RTs increase as a function of word length by approximately 26 ms per letter. This is the overall effect, but it's likely that the effect varies across participants. First, let's try to do some plots to understand this better.

First, plot the overall effect of word length on RTs. Use the `stat_summary()` command to summarize the Y variable for each X value. Then use the `geom_smooth(method='lm', se=F)` command to plot a linear regression line through the data.

In [ ]:
# insert code here

Now, let's see how much this effect varies across participants. Make the same plot, but this time in the `aes()` function add the argument `color=Sub_ID`. This will create separate regression lines in different colors for each participant.

In [ ]:
# insert code here

Describe briefly the plot above. What stands out to you?

> insert response here

The plot above has too manyt subjects so it's a bit difficult to see them well. Just for the exercize, let's do the same plot but for a smaller subset of subjects. Filter the data so that you keep only data for subjects 8,13,50,56,62,78,82 and repeat the plot above with this filtered dataset. Which of these subjects has the lowest intercept? Which one seems to have the steepest slope?

In [ ]:
# insert code here

#### Part 3: Fit separate linear regression to each subject

Before we fit a mixed-effects model, let's pretend that each subject is it's own experiment and let's fit a separate linear regression for each subject. We will then extract the intercepts and slopes for each participant's regression, and plot their distribution. The next few cells will take you step by step through this process

Create a new variable called `sub_ids` that contains the unique values of Sub_ID. This variable should have 99 unique values (you can use the `unique()` command):

In [ ]:
# insert code here

Now we need to create a `for` loop that will fit a regression separately for each participant. We can do that step by step. Please read https://www.datacamp.com/community/tutorials/tutorial-on-loops-in-r if you are not familiar with how `for` loops work. We will create the following basic algorithm:

1. Create an empty variable to store the intercepts for each subject
2. Create an empty variable to store the slopes for each subject
3. [Loop] - for each value of sub_ids:
    - Extract the data for that subject and assign it to a temporary variable
    - Fit a regression model to the extracted data and assign the output to a temporary variable
    - Extract the intercept from the model variable, and append it to the empty intercept variable we created before
    - Extract the slope from the model variable and append it to the empty slope variable we created before

Since this is a fairly advanced use of R programming, the basic code for this is provided below, but you need to fill out some missing details:
- You will need to replace the `df1` variable with the name of your main data.frame that contains data for all subjects
- In the line `sub_ml <- ` add a linear regression of D_RT as a function of Length. Use the `sub_data` in the linear regression call as an argument for `data = `


Then run the cell below:

In [ ]:
sub_intercepts <- c() # create an empty variable to store the intercepts for each subject
sub_slopes <- c() # create an empty variable to store the slopes for each subject

# loop over sub_id values
for (subject in sub_ids) {
    sub_data <- filter(df1, Sub_ID == subject)  # extract data for the subject in each loop step
    sub_ml <- lm(D_RT ~ Length, data=sub_data)  # run a regression for each subjects data
    intercept <- coef(sub_ml)[1] # extract the intercept for the subject in each loop step
    slope <- coef(sub_ml)[2] # extract the slope for the subject in each loop step
    sub_intercepts <- c(sub_intercepts, intercept) # append the intercept of this subject to the vector of intercepts for each subject
    sub_slopes <- c(sub_slopes, slope) # append the intercept of this subject to the vector of intercepts for each subject
}

Now, both `sub_intercepts` and `sub_slopes` should be vectors with 99 values each. Show their contents in the cell below:

In [ ]:
# insert code here

Let's combine `sub_ids`, `sub_intercepts` and `sub_slopes` in a new data.frame with 3 columns named `id`, `intercept` and `slope` that correspond to each of these variables. The data.frame should have 99 rows, one row for each subject. Assign the result to a new variable called `sub_lm_coefs` and show it's contents:

In [ ]:
# insert code here

Use the `arrange()` command from the `tidyverse` package to sort the data frame first by the intercept and then by the slope. Answer the following questions:
- which subject has the slowest and which has the fastest intercept?
- which subject has the steepest and the least steep slope? Explain their slope values in terms of how word length affects RTs
- are there any subject for which Length affects RTs negatively?

In [ ]:
# insert code for intercept here

> insert response here

In [ ]:
# insert code for slope here

> insert response here

Make 3 plots:
- A histogram of the intercepts for each participant
- A histogram of the slopess for each participant
- A scatterplot of the relationship between intercepts and slopes for each participant

In [ ]:
# insert code here

Are the intercepts and slopes normally distributed across subjects?

> insert response here

Are the intercept and slope for each subject correlated? That is, if the subject is overall slower in the task, are they more strongly affected by the word length property?

In [ ]:
# insert code here

> insert response here

What is the mean intercept and slope over all participants?

In [ ]:
# insert code here

#### Part 4: Fit a mixed-effects regression to the data

We just fit a separate regression model to the data from each participant, ignoring the data from other participants. This accounts for individual differences, but it has several drawbacks:
- it is not easy to test directly whether overall the effect of word length has an effect of RTs. The fact that all subjects showed positive slopes is an indication that this effect is overall present, but we need a more systematic way to do this
- in fitting the data of each subject, we completely ignore data from other subjects. Even though individual differences are expected, it is also reasonable to expect that people cognitive processes are similar. Thus, we are throwing away a lot of data when doing this, because we assume that one participants' parameters are not informed by data from other participants. Since we have a limited number of observations per participant, the individual parameter estimates will be more noise and more easily affected by outliers. This can lead to a biased estimation.

Both of these drawbacks can be overcome by fitting a mixture effects regression, which is a compromise between completely pooling the data, which ignores individual differences (e.g., Homework 4), and the completely separate regression for each participant, which ignores commonalities between participants and throws away data.

First, install and load the `lme4` package, which allows us to fit mixed effects regressions:

In [ ]:
# insert code here

Now, use the `lmer()` command to fit a simple mixed-effects linear regression model with only random intercepts for each participant (consult the tutorials you read last week for how to do this. This should be a basic regression of D_RT with only an intercept. You can use the formula `D_RT ~ 1 + (1|Sub_ID)`. Save the output to a variable called `mixed_ml0` and use the `summary()` command to display the results

In [ ]:
# insert code here

This simple model does not have predictors. In this case, the Intercept is merely the mean RTs overall, or what we would expect the RTs to be if all predictors are set to their mean value.

Now, let's add the effect of word length as a fixed effect to the formula. Save the results to a variable `mixed_ml1` and use `summary(mixed_ml1)` to display the results

In [ ]:
# insert code here

What are the overall intercept and slope for Length? How do you interpret these values?

As you notice, there are no p-values in this output. We can use the command "anova(mixed_ml0, mixed_ml1) to perform a likelihood ratio test, and to use the Aikaike Information Criterion (AIC) to see if the mixed_ml1 model fits the data better than the mixed_ml0 model:

In [ ]:
# insert code here

You see that the output has two rows, one for each model. The `npar` column shows the number of parameters in the model. The AIC and BIC columns show two different indices of model fit. The best fitting model is the one with the lowest AIC. If the more complex model (ml1) has an AIC that is lower by more than 2 than the next best fitting model, we can conclude that including the Length variable improves the model. The table also provides a p-value (in scientific number notation) for the comparison between models. In this case, mixed_ml1 fits significantly better than the mixed_ml0, so word length improves the model fit and should be kept

We have only fit random intercepts for each subject, which assumes that each subject has a different baseline of RTs, but that each subject is affectedf similarly by word Length. You can extract the random effect parameters for each subject with the following command:

In [ ]:
ml1_coefs <- coef(mixed_ml1)$Sub_ID
names(ml1_coefs) <- c('intercept','slope')
ml1_coefs

This table is similar to the one we constructed manually from the separate regressions; however, the slope is the same for all participants. Let's fit the model again, this time by including random slopes for Length. You can do that with the formula `D_RT ~ Length + (Length|Sub_ID)`. Save the result to the `mixed_ml2` variable and show the results using `summary()`

In [ ]:
# insert code here

What are the overall intercept and slope for Length? Have they changed much relative to the model without a random slope?

> insert response here

You can use the `anova()` command again to test whether the random effect improves the model fit:

In [ ]:
# insert code here

Now, finally, let us extract the random effects from this model, and compare them to the separate intercept and slopes we fit to each participant in the previous part. First, use the `coef()` command on the final model, and save the output to a variable called `mixed_lm_coefs`. Repeat all steps I showed you above in order to rename the columns with more useful names

In [ ]:
# insert code here

Repeat the plots we made in the previous part, this time using the mixed-effects random effects:
- A histogram of the intercepts for each participant
- A histogram of the slopess for each participant
- A scatterplot of the relationship between intercepts and slopes for each participant

In [ ]:
# insert code here

What differences do you notice relative to the plots for the separately fit regressions? Pay attention to the values on the x-axis for the histograms. Why do you think these differences occur?

> insert response here

In the next couple of plots, I'm plotting the parameter estimates from the mixed effects model relative to the parameter estimates from the separate by-subject regressions. Each dot is a participant. The black line represents a diagonal on which parameter estimates would be equal in the two models - points under the diagonal have smaller parameter estimates in the mixed-effects model. Points above the diagonal have a larger parameter estimates in the mixed-effects regression relative to the separate by-subject regressions. What are the main patterns you notice in these plots? Can those patterns help you answer the previous question?

In [ ]:
sub_lm_coefs <- arrange(sub_lm_coefs, id)
both_coefs <- bind_cols(sub_lm_coefs, mixed_lm_coefs)
names(both_coefs) <- c('id','intercept_separate','slope_separate', 'intercept_mixed','slope_mixed')

ggplot(both_coefs, aes(intercept_separate, intercept_mixed)) +
    geom_point() +
    geom_abline(intercept=0, slope=1) +
    coord_cartesian(xlim=c(300,1000), ylim=c(300, 1000))


ggplot(both_coefs, aes(slope_separate, slope_mixed)) +
    geom_point() +
    geom_abline(intercept=0, slope=1) 


> insert response here